# Databricks SQL Alerts

**Objective:**
Learn how to monitor your data and business metrics using **Databricks SQL Alerts**. We will set up a mechanism to send notifications to specific destinations (like Email, Slack, or Teams) when a specific condition in your data is met.

### What are SQL Alerts?
SQL Alerts allow you to run a SQL query periodically, evaluate the result against a defined condition, and send notifications if that condition is satisfied.

**Common Use Cases:**
1.  **Infrastructure Monitoring:** Alert if a sensor status is 'DOWN'.
2.  **Data Quality:** Alert if `NULL` count in a critical column exceeds a threshold.
3.  **Business Logic:** Alert if "Pending Orders" count exceeds 10,000 (The use case for today).

---
### Prerequisites
*   You must have access to **Databricks SQL**.
*   We will use the `orders_raw` table created in previous sessions (located in `dev.bronze`).
*   **Note:** Creating "Destinations" requires **Workspace Admin** privileges.

## Step 1: Create the SQL Query

The foundation of an alert is a SQL Query. The alert will execute this query and check the output.

**Scenario:** We want to be alerted if the number of orders with status **'P' (Pending)** exceeds **10,000**.

Run the following query in your SQL Editor (or here to test) to verify the data.

In [ ]:
-- This query calculates the count of pending orders
-- We will save this query in the SQL Editor as "Order_Pending_Count"

SELECT count(1) as pending_orders_count 
FROM dev.bronze.orders_raw 
WHERE o_orderstatus = 'P';

## Step 2: Configure the Alert (UI Steps)

Since Alerts are configured via the Databricks SQL UI, follow these steps:

1.  **Navigate to Alerts:** Click on **Alerts** in the left sidebar of the Databricks SQL persona.
2.  **Create Alert:** Click the **Create Alert** button.
3.  **Select Query:** Search for and select the query we created in Step 1 (`Order_Pending_Count`).
4.  **Define Trigger Condition:**
    *   **Value Column:** `pending_orders_count` (the output column of our query).
    *   **Operator:** `>` (Greater than).
    *   **Threshold Value:** `10000`.
    *   *Logic:* If `pending_orders_count > 10000`, the alert status becomes `TRIGGERED`.
5.  **Empty Results:** Set "When query result has no rows" to **OK** (implies 0 pending orders).

## Step 3: Customizing the Notification Template

You can customize the message sent to the destination using variables.

1.  Under **Template**, select **Use custom template**.
2.  You can use variables like `{{QUERY_RESULT_VALUE}}` to inject dynamic data into the message.

**Example Subject:**
`Pending Orders Alert`

**Example Body:**

Attention Team,

The number of pending orders has exceeded the threshold.
Current Count: {{QUERY_RESULT_VALUE}}

Please investigate immediately.

## Step 4: Configure Destinations

Destinations define *where* the notification is sent. 
*   *Note: Only Workspace Admins can create new Destinations.*
*   **Supported Destinations:** Email, Slack, Webhook, MS Teams, PagerDuty.

**To Create a Destination:**
1.  Go to **Settings** (Profile Icon) -> **Settings**.
2.  Select **Notifications** tab.
3.  Click **Manage** next to Destinations.
4.  Add a new destination (e.g., Email) and enter the address (e.g., `user@example.com`).

**Link Destination to Alert:**
1.  Go back to your Alert configuration.
2.  Under **Destinations**, select the destination you just created.

## Step 5: Scheduling

An Alert needs its own schedule to run. It does **not** rely on the schedule of the underlying Query.

1.  Click **Add Schedule** in the Alert view.
2.  Set the frequency (e.g., "Every 1 Hour").
3.  **SQL Warehouse:** Ensure a SQL Warehouse is selected to execute the check.

**Important Note:** 
The SQL Warehouse attached to the Alert query will be used to evaluate the alert. Ensure this warehouse is running or has auto-start enabled when the schedule triggers.

In [ ]:
# Practical Validation
# While we configured this in the UI, we can use PySpark here to verify if our data currently meets the condition.

df = spark.sql("SELECT count(1) as pending_orders_count FROM dev.bronze.orders_raw WHERE o_orderstatus = 'P'")
count = df.collect()[0]['pending_orders_count']

print(f"Current Pending Orders: {count}")

if count > 10000:
    print("Condition Met: If the Alert was active, it would be TRIGGERED.")
else:
    print("Condition Normal: Alert would be OK.")